In [1]:
from pathlib import Path
import sys

ROOT = Path().resolve().parents[1] / "code"
sys.path.append(str(ROOT))


In [2]:
import polars as pl
from functions.read_news import read_news
from functions.read_behaviors import read_train_behaviors,read_dev_behaviors
from functions.valid_recall import valid_popularity_recall

In [3]:
actions=read_train_behaviors()

In [4]:
current=actions['time'].max()
current

datetime.datetime(2019, 11, 14, 23, 59, 59)

In [5]:
click=actions.select(['time','impressions']).explode('impressions').with_columns([
    pl.col('impressions').str.split('-').list.get(0).alias('new_id'),
    pl.col('impressions').str.split('-').list.get(1).cast(pl.UInt8).alias('label')]
).filter(pl.col('label')==1)

In [6]:
del actions

In [7]:
click=click.drop('impressions')
click.head()

time,new_id,label
datetime[μs],str,u8
2019-11-10 11:30:54,"""N94157""",1
2019-11-10 11:30:54,"""N78699""",1
2019-11-10 11:30:54,"""N71090""",1
2019-11-10 11:30:54,"""N31174""",1
2019-11-12 13:45:29,"""N25587""",1


In [8]:
del click

In [9]:
def processData(actions):

    current=actions['time'].max()

    click=actions.select(['time','impressions']).explode('impressions').with_columns([

    pl.col('impressions').str.split('-').list.get(0).alias('new_id'),
    pl.col('impressions').str.split('-').list.get(1).cast(pl.UInt8).alias('label')]
).filter(pl.col('label')==1)
    click=click.drop('impressions')

    return click,current

In [10]:
def popularityNews(path,window=1, topk=50):
    """
    data: click dataframe (new_id, time)
    current_time: 当前时间 (Datetime)
    window: 时间窗口（天）
    topk: 召回数量
    """

    if path:actions=pl.read_parquet(path)
    else:actions=read_train_behaviors()

    data,current_time=processData(actions)
    window_seconds = window * 86400

    recent = data.filter(
        pl.col("time") >= current_time - pl.duration(seconds=window_seconds)
    )

    popular = (
        recent
        .group_by("new_id")
        .agg(
            pl.count().alias("click_cnt")
        )
        .sort("click_cnt", descending=True)
        .head(topk)
    )

    return popular

In [11]:
def recall_popularity(train_path,pred_path,window=1, topk=50):
    popular = popularityNews(train_path,window, topk)

    rec_list = popular.select(
        pl.col("new_id").alias("rec_list")
    ).to_series().to_list()

    if pred_path:
        users = pl.read_parquet(pred_path).select("user_id").unique()
    else:
        users = read_dev_behaviors().select("user_id").unique()

    result = users.with_columns(
        pl.lit(rec_list).alias("rec_list")
    )

    return result

In [12]:
res=recall_popularity(None,None,window=0.5,topk=50)

/tmp/ipykernel_63056/3034725002.py:23: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("click_cnt")


In [13]:
valid_popularity_recall(res)

Recall@10 0.005779432411389811
Recall@20 0.021306511849327085
Recall@30 0.05903274772229882
Recall@40 0.06472932223549374
Recall@50 0.07062445904476734


In [14]:
windows = [0.25,0.5,0.6,0.7,1,1.5,2,3]

In [15]:
for w in windows:
    res=recall_popularity(None,None,window=w,topk=50)
    print(f'recall window is {w}')
    valid_popularity_recall(res)
    print()

/tmp/ipykernel_63056/3034725002.py:23: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("click_cnt")


recall window is 0.25
Recall@10 0.04846600830711261
Recall@20 0.06099433714110569
Recall@30 0.0698278925479939
Recall@40 0.07511358199932348
Recall@50 0.18459737554662395

recall window is 0.5
Recall@10 0.005779432411389811
Recall@20 0.02130651184932708
Recall@30 0.05903274772229882
Recall@40 0.06472932223549374
Recall@50 0.07062445904476737

recall window is 0.6
Recall@10 0.005713899771117523
Recall@20 0.015836581776318467
Recall@30 0.025546719572444542
Recall@40 0.06498404026593813
Recall@50 0.06770441785714967

recall window is 0.7
Recall@10 0.004188536356324833
Recall@20 0.00868583713361874
Recall@30 0.01776116417866092
Recall@40 0.02909973212227942
Recall@50 0.03170982669926206

recall window is 1
Recall@10 0.0037118274890835575
Recall@20 0.0063444948150216006
Recall@30 0.009889817744460612
Recall@40 0.016812976528223007
Recall@50 0.029376368208820707

recall window is 1.5
Recall@10 0.003500121720687066
Recall@20 0.00590607782807503
Recall@30 0.007305686119273664
Recall@40 0.01208

In [16]:
res=recall_popularity(None,None,window=0.5,topk=50)
res.write_parquet('recall_results/popularity_half_day.parquet')

/tmp/ipykernel_63056/3034725002.py:23: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("click_cnt")
